In [1]:
import numpy as np
import pandas as pd
import json
import tweepy
import seaborn as sns
import os
import requests as req
import calendar
import pytz
import time
import got3

from datetime import datetime, date, timedelta

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()



In [2]:
api_dir = os.path.dirname(os.path.dirname(os.path.realpath('keys')))
file_name = os.path.join(api_dir + "//keys", "api_keys.json")
data = json.load(open(file_name))

gkey = data['google_api_key']
consumer_key = data['twitter_consumer_key']
consumer_secret = data['twitter_consumer_secret']
access_token = data['twitter_access_token']
access_token_secret = data['twitter_access_token_secret']


In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())



In [4]:
#coin list by symbol in descending order of market cap
#bitcoin - ethereum - ripple - bitcoin cash - cardano - litecoin - stella - neo - eos - xem  
coin_list = ['BTC', 'ETH', 'XRP', 'BCH', 'ADA', 'LTC', 'XLM', 'NEO', 'EOS', 'XEM']



In [5]:
#URLs
social_stats_url = "https://www.cryptocompare.com/api/data/socialstats/?id="

coin_list_url = "https://www.cryptocompare.com/api/data/coinlist/"

coin_historical_url = "https://min-api.cryptocompare.com/data/pricehistorical?fsym="
coin_historical_url_second = "&tsyms=USD&ts="



In [6]:
#variables 
coin_list_response = []

coin_data_df = pd.DataFrame(columns=["symbol", "date", "coin_name", "coin_price", 
                                     "compound_average",
                                     "positive_average", "neutral_average", "negative_average"
                                   ])



In [7]:
def GetCoinList():
    response = req.get(coin_list_url).json()

    count = 0
    for each in response["Data"]:
        count = count + 1
    
    print(count)
    return response


def GetCoinId(symbol):
    for each in coin_list_response["Data"]:
        row = coin_list_response["Data"][symbol]
        if row["Name"] == symbol:
            return int(row["Id"])


def GetCoinName(symbol):
    for each in coin_list_response["Data"]:
        row = coin_list_response["Data"][symbol]
        if row["Name"] == symbol:
            name = row["CoinName"].lower().split(" / ")
            return name[0]

               
def GetHistoricalPrice(symbol, utctime):
    #url to get historical price
    url = coin_historical_url + symbol + coin_historical_url_second + str(utctime)
    #get the historical price
    price = req.get(url).json()
    return float(price[symbol]["USD"])


def GetSocialStats(coin_id):
    url = social_stats_url + coin_id 
    return req.get(url).json()


def GetUtcTime(year,month,day):
    utc = calendar.timegm(datetime(year,month,day).utctimetuple())
    return utc

    
def GetLastDayOfMonth(year,month,day):
    d = date(year,month,day)
    a = date(d.year + (d.month == 12),(d.month + 1 if d.month < 12 else 1), 1) - timedelta(1)
    return a.day


def AnalyzeTweets(coin_name, until_date):
   
    tweetCriteria = got3.manager.TweetCriteria().setUntil(until_date).setQuerySearch(coin_name).setMaxTweets(max_tweets)
    
    for i in range(max_tweets):
        #each tweet
        tweets = got3.manager.TweetManager.getTweets(tweetCriteria)
        if tweets is None:
            break
        else:
            try:
                tweet = tweets[i] 
                #run Vader analysis
                compound = analyzer.polarity_scores(tweet.text)["compound"]
                pos = analyzer.polarity_scores(tweet.text)["pos"]
                neu = analyzer.polarity_scores(tweet.text)["neu"]
                neg = analyzer.polarity_scores(tweet.text)["neg"]
                #add each score to an appropriate array
                compound_list.append(compound)
                positive_list.append(pos)
                negative_list.append(neg)
                neutral_list.append(neu)
            except Exception: 
                print("No more tweets available")
                break




In [8]:
#PLEASE CHANGE THIS BEFORE EXECUTING THE CODE
year = 2017
month = 12
day = 1

last_day_of_month = GetLastDayOfMonth(year,month,day)

#sampling max number of tweets per day per coin
max_tweets = 25

In [9]:
#time marker
begin = str(datetime.now())


In [10]:
#get coin list
coin_list_response = GetCoinList()

count = 0

#build coin data
while count < last_day_of_month:
    #get UTC time, one day at a time
    utc_time = GetUtcTime(year,month,day)
    until_date = date(year,month,day).strftime("%Y-%m-%d")  
 
    #process one coin at a time
    for symbol in coin_list:
        coin_name = GetCoinName(symbol)
        
        #calculate Vader scores
        compound_list = []
        positive_list = []
        negative_list = []
        neutral_list = []
        AnalyzeTweets(coin_name,until_date)
        
        row = {"symbol": symbol, 
               "date": until_date,
               "coin_name": coin_name,
               "coin_price": GetHistoricalPrice(symbol,utc_time),
               "compound_average": float(np.mean(compound_list)),
               "positive_average": float(np.mean(positive_list)), 
               "neutral_average": float(np.mean(neutral_list)), 
               "negative_average": float(np.mean(negative_list)) }
        
        coin_data_df = coin_data_df.append(row, ignore_index=True)   

        print(until_date," ",symbol," ",coin_name, np.mean(compound_list))

    count = count + 1
    
    #get the next date
    next_date = date(year,month,day) + timedelta(1)
    year = next_date.year
    month = next_date.month
    day = next_date.day
    
coin_data_df

2112
2018-01-01   BTC   bitcoin 0.140736
2018-01-01   ETH   ethereum 0.23534
2018-01-01   XRP   ripple 0.417144
2018-01-01   BCH   bitcoin cash 0.078208
2018-01-01   ADA   cardano 0.215136
2018-01-01   LTC   litecoin 0.055248
2018-01-01   XLM   stellar 0.289116
2018-01-01   NEO   neo 0.114784
2018-01-01   EOS   eos 0.094384
2018-01-01   XEM   nem -0.068564
2018-01-02   BTC   bitcoin 0.130088
2018-01-02   ETH   ethereum 0.26132
2018-01-02   XRP   ripple 0.151636
2018-01-02   BCH   bitcoin cash 0.096796
2018-01-02   ADA   cardano 0.171904
2018-01-02   LTC   litecoin 0.101792
2018-01-02   XLM   stellar 0.140672
2018-01-02   NEO   neo 0.077868
2018-01-02   EOS   eos 0.054032
2018-01-02   XEM   nem -0.01184
2018-01-03   BTC   bitcoin 0.180132
2018-01-03   ETH   ethereum 0.11626
2018-01-03   XRP   ripple 0.141772
2018-01-03   BCH   bitcoin cash 0.244168
2018-01-03   ADA   cardano 0.16784
2018-01-03   LTC   litecoin 0.148976
2018-01-03   XLM   stellar 0.192352
2018-01-03   NEO   neo 0.074608


2018-01-24   XRP   ripple 0.087172
2018-01-24   BCH   bitcoin cash 0.052476
2018-01-24   ADA   cardano 0.19016
2018-01-24   LTC   litecoin 0.084328
2018-01-24   XLM   stellar 0.178072
2018-01-24   NEO   neo -0.013324
2018-01-24   EOS   eos 0.137772
2018-01-24   XEM   nem -0.004788
2018-01-25   BTC   bitcoin 0.118008
2018-01-25   ETH   ethereum 0.089776
2018-01-25   XRP   ripple 0.145872
2018-01-25   BCH   bitcoin cash 0.15372
2018-01-25   ADA   cardano 0.097844
2018-01-25   LTC   litecoin 0.224532
2018-01-25   XLM   stellar 0.3458
2018-01-25   NEO   neo 0.176784
2018-01-25   EOS   eos 0.100192
2018-01-25   XEM   nem 0.021956
2018-01-26   BTC   bitcoin -0.036636
2018-01-26   ETH   ethereum 0.185484
2018-01-26   XRP   ripple 0.000728
2018-01-26   BCH   bitcoin cash 0.138628
2018-01-26   ADA   cardano 0.073072
2018-01-26   LTC   litecoin 0.299636
2018-01-26   XLM   stellar 0.271992
2018-01-26   NEO   neo 0.016636
2018-01-26   EOS   eos 0.195344
2018-01-26   XEM   nem 0.057728
2018-01-27  

,symbol,date,coin_name,coin_price,compound_average,positive_average,neutral_average,negative_average
0,BTC,2018-01-01,bitcoin,13444.8800,0.140736,0.08964,0.87804,0.03220
1,ETH,2018-01-01,ethereum,756.2000,0.235340,0.10588,0.88388,0.01024
2,XRP,2018-01-01,ripple,2.0500,0.417144,0.20968,0.77628,0.01408
3,BCH,2018-01-01,bitcoin cash,2319.1200,0.078208,0.07496,0.86408,0.06100
4,ADA,2018-01-01,cardano,0.7022,0.215136,0.10432,0.87684,0.01880
5,LTC,2018-01-01,litecoin,224.3400,0.055248,0.05852,0.91404,0.02740
6,XLM,2018-01-01,stellar,0.4840,0.289116,0.19256,0.73564,0.03180
7,NEO,2018-01-01,neo,78.0900,0.114784,0.05668,0.88508,0.01824
8,EOS,2018-01-01,eos,7.6400,0.094384,0.03540,0.95380,0.01084
9,XEM,2018-01-01,nem,1.0100,-0.068564,0.01332,0.93676,0.04992


In [11]:
file_name = os.path.join("resources", "coin_tweet_analysis_2017_12.csv")
coin_data_df.to_csv(file_name)


In [12]:
#time marker
print("begin...",begin)
print("end.....",str(datetime.now()))

begin... 2018-02-04 21:44:14.339314
end..... 2018-02-05 01:17:56.315851
